# **Requisitos**

Antes de executar este notebook, verifique os seguintes pontos:

### **1. Arquivos RAW gerados**
Certifique-se de que, ao rodar o arquivo **ETL.ipynb**, foram gerados corretamente os seguintes arquivos:

- `DENGBR20.csv`  
- `DENGBR21.csv`  
- `DENGBR22.csv`

Esses arquivos serão usados na etapa de **extração** dos dados.

---

### **2. Banco de dados PostgreSQL configurado**
É necessário ter o **PostgreSQL** instalado e uma **database** criada especificamente para este projeto.

Você deve saber e guardar as seguintes informações para configurar a conexão:

- **Usuário**  
- **Senha**  
- **Host** (em geral: `localhost`)  
- **Porta** (padrão: `5432`)  
- **Nome do banco**  

Essas credenciais serão usadas para estabelecer a comunicação entre o notebook e o banco de dados via SQLAlchemy.


## **Instale as dependências**

Usaremos pandas para ler os arquivos brutos .csv e sqlalchemy para criar a conexão com o banco de dados postgreSQL.

In [1]:
pip install pandas sqlalchemy psycopg2-binary

Note: you may need to restart the kernel to use updated packages.


## **Configuração da Conexão e Definição dos Arquivos RAW**

Nesta etapa, realizamos duas ações fundamentais para iniciar o processo de ingestão dos dados:

### **1. Configuração da conexão com o PostgreSQL**
Aqui definimos:
- Usuário, senha, host, porta e nome do banco  
- A `DATABASE_URL`, que monta a string de conexão  
- O `engine` do SQLAlchemy, responsável por estabelecer a comunicação entre Python e PostgreSQL  

Com isso, garantimos que o notebook consiga enviar dados para o banco.

---

### **2. Mapeamento dos arquivos RAW**
Criamos um dicionário chamado `RAW_DATA_FILES`, onde cada chave representa o nome da tabela bruta no banco, e cada valor corresponde ao arquivo CSV bruto que será lido.


In [ ]:
import pandas as pd
from sqlalchemy import create_engine


#1. Configuração da conexão com o PostgreSQL

DB_USER = 'postgres'
DB_PASSWORD = 'password'
DB_HOST = 'localhost'
DB_PORT = '5432'
DB_NAME = 'dengue_db'

DATABASE_URL = f'postgresql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}'
engine = create_engine(DATABASE_URL)

print("Conexão com o banco de dados estabelecida com sucesso.")


# 2. Arquivos RAW

RAW_DATA_FILES = {
    'dengue_raw_2020': 'DENGBR20.csv',
    'dengue_raw_2021': 'DENGBR21.csv',
    'dengue_raw_2022': 'DENGBR22.csv'
}



Conexão com o banco de dados estabelecida com sucesso.


### **ETAPA DE EXTRACT E LOAD (EL)**

Este bloco executa a ingestão dos dados RAW seguindo duas etapas principais:


### **1. Extract (Extração)**

- Lê cada arquivo CSV bruto listado no dicionário `RAW_DATA_FILES`.
- A leitura é feita **em chunks (lotes)**, definidos pela constante `CHUNKSIZE` (ex.: 10.000 linhas por vez).
- Os dados são carregados **sem transformações**, preservando o formato original.
- Linhas problemáticas são ignoradas com `on_bad_lines='skip'`.
- Todos os valores são lidos como texto (`dtype=str`) para evitar erros de tipo durante a carga.


### **2. Load (Carga)**

- Cada chunk é inserido diretamente em uma tabela no PostgreSQL.
- O nome da tabela é definido pela chave do dicionário `RAW_DATA_FILES`.
- **Apenas o primeiro chunk** usa `if_exists='replace'`, recriando a tabela no início da carga.
- Os **chunks seguintes** usam `if_exists='append'`, adicionando os dados de forma incremental.
- O `to_sql()` utiliza `method='multi'` para otimizar a inserção enviando várias linhas por comando SQL.
- O parâmetro `chunksize` também é aplicado na escrita para melhorar o desempenho.


In [3]:
import pandas as pd

print("Iniciando processo de ingestão dos dados RAW...\n")

CHUNKSIZE = 10_000  # qqr coisa muda o tamanho do chunk aqui

for table_name, file_path in RAW_DATA_FILES.items():
    try:
        print(f"\nLendo arquivo em chunks: {file_path}")
        first_chunk = True

        for chunk in pd.read_csv(
            file_path,
            encoding='utf-8',
            on_bad_lines='skip',
            chunksize=CHUNKSIZE,
            dtype=str 
        ):

            print(f"Inserindo {len(chunk)} registros na tabela '{table_name}'...")


            chunk.to_sql(
                table_name,
                con=engine,
                if_exists='replace' if first_chunk else 'append',
                index=False,
                chunksize=CHUNKSIZE,
                method='multi'
            )

            first_chunk = False

        print(f"Tabela '{table_name}' carregada com sucesso.\n")

    except Exception as error:
        print(f"\nERRO ao processar '{file_path}': {error}\n")

print("Ingestão dos dados RAW finalizada com sucesso.")


Iniciando processo de ingestão dos dados RAW...


Lendo arquivo em chunks: DENGBR20.csv
Inserindo 10000 registros na tabela 'dengue_raw_2020'...
Inserindo 10000 registros na tabela 'dengue_raw_2020'...
Inserindo 10000 registros na tabela 'dengue_raw_2020'...
Inserindo 10000 registros na tabela 'dengue_raw_2020'...
Inserindo 10000 registros na tabela 'dengue_raw_2020'...
Inserindo 10000 registros na tabela 'dengue_raw_2020'...
Inserindo 10000 registros na tabela 'dengue_raw_2020'...
Inserindo 10000 registros na tabela 'dengue_raw_2020'...
Inserindo 10000 registros na tabela 'dengue_raw_2020'...
Inserindo 10000 registros na tabela 'dengue_raw_2020'...
Inserindo 10000 registros na tabela 'dengue_raw_2020'...
Inserindo 10000 registros na tabela 'dengue_raw_2020'...
Inserindo 10000 registros na tabela 'dengue_raw_2020'...
Inserindo 10000 registros na tabela 'dengue_raw_2020'...
Inserindo 10000 registros na tabela 'dengue_raw_2020'...
Inserindo 10000 registros na tabela 'dengue_raw_2020'...
